In [1]:
# This notebook is  used to evaluate perofrmance of our RAG ingestion and query pipeline.
# Method:
#   - Generated test data using a sample GitHub repository (offline process not in this notebook)
#     - https://docs.ragas.io/en/stable/getstarted/rag_testset_generation/
#   - Use our ingestion pipeline to parse the same reporisiroty and index in our vector store
#   - Use RAGAS on teadt dataset + our answers fromn our RAG and visualise the mterics.
import os
import sys
import pandas as pd

sys.path.insert(1, '/home/jovyan/work/code')
from config import VectorDBConfig, EmbeddingConfig, ProcessingConfig, ChatConfig
from config_helper import ConfigHelper
from pipeline import DocumentPipeline
from localrag import LocalRAG

from TraceSetup import get_tracer, get_logger
logger = get_logger()
tracer = get_tracer()

config_helper = ConfigHelper(False)

In [ ]:
# Ingest a GitHub repository and import into our vector store

# Initialize the pipeline
pipeline = DocumentPipeline(
    vector_db_config=config_helper.vector_db_config,
    embedding_config=config_helper.embedding_config,
    logger=logger,
    tracer=tracer)

local_ingestion_file="dotnet-docs-aspire.txt"
repository="https://github.com/dotnet/docs-aspire"
with tracer.start_as_current_span(f"Starting ingesting {repository} using local ingestion file {local_ingestion_file}"):
    pipeline.process_single_file(local_ingestion_file,repository)


In [2]:
# Test our RAG Solution

def demonstrate_local_rag(rag):
    """Demonstrate how to use the LocalRAG class."""    
    # Example questions to test
    questions = [
        "If I use Docker Compose, why should I care about .Net Aspire?",
        "Is .Net Aspire an alternative to Kubernetes?"
    ]
    with tracer.start_as_current_span("Entering questions loop."):
        for question in questions:
            print(f"Question: {question}")
            # print("\nRelevant chunks:")
            # with tracer.start_as_current_span("rag get chunks"):
            #     chunks = rag.get_relevant_chunks(question, k=5)
            #     for i, chunk in enumerate(chunks, 1):
            #         print(chunk)
            #         break
#                    print(f"\nChunk {i}:")
#                    print(f"Source: {chunk.metadata.get('file_path', 'Unknown')}")
#                    print(f"Content: {chunk.page_content[:200]}...")
                    
            print("\nGenerated Answer:")
            with tracer.start_as_current_span("Retrieve answers."):
                answer = rag.retrieve_and_answer(question, k=10)
                print(answer)
                print("\n" + "="*80 + "\n")

rag = LocalRAG(
    vector_db_config=config_helper.vector_db_config,
    embedding_config=config_helper.embedding_config, 
    chat_config=config_helper.chat_config,
    logger=logger,
    tracer=tracer)

with tracer.start_as_current_span("Starting demo"):
    demonstrate_local_rag(rag)

/home/jovyan/work/code/localrag.py:43: UserWarning: Api key is used with an insecure connection.
  self.qdrant = QdrantClient(url=vector_db_config.url, api_key=vector_db_config.api_key)


Question: If I use Docker Compose, why should I care about .Net Aspire?

Generated Answer:
Using `.NET Aspire` offers several advantages over relying solely on `Docker Compose`, particularly in terms of development experience and orchestration capabilities. Here are the reasons:

1. **Developer Experience** - Unlike Docker Compose, which may become complex with abstractions or composition tasks (refer to eshopOnContainers), `.NET Aspire` is designed for ease-of-use without requiring additional learning beyond what you already know about .NET development and containerization (`Docker Desktop/Podman`). The declarative code used in `.NET Aspire`, as opposed to YAML files, can lead to a more intuitive configuration process with better readability when handling environment variables (replacements) and includes no types or IntelliSense by default for easier understanding of the running application components (`Debugging difficulties`).

2. **Containerization** - `.NET Aspire` projects are ex